In [1]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_verbose

set_verbose(True)
model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("說出一個關於'{topic}'的笑話")

chain = prompt | model | StrOutputParser()

In [3]:
chain.invoke({"topic": "軟體工程師"})

'為什麼軟體工程師總是在午夜時分工作？\n因為他們喜歡在「黑暗中」編程！'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_verbose
from langchain_core.prompts import ChatPromptTemplate

system = """你是一個專業的心理學家，你擅長分析日記中所包含的情緒。 \
你將需要根據使用者的日記內容分析其情緒，並將其代表的顏色以及對應的內容段落，分別放進名為 "color" 和 "content" 的 key 中，並以 JSON 格式回傳。 \
注意：你最少需要挑選一個顏色，並且最多挑選不超過兩個顏色，請挑選日記中最具有代表性的段落。 \
以下是 8 種不同的情緒所對應的顏色: \
1. 憤怒暴躁: Red \
2. 快樂喜悅: Yellow \
3. 悲傷難過: Blue \
4. 恐懼害怕: Purple \
5. 焦慮不安: Orange \
6. 厭惡煩躁: Green \
7. 平靜祥和: Indigo \
8. 其他 (當日記不包含任何情緒或是沒有內容時): Gray \

以下是幾個範例:

example_user: 今天的工作非常緊張，一整天都在忙碌中度過。午休時，莉莉邀請我週末一起去看電影，我感到非常開心。下班後，坐公交回家，一路上都在想著週末的計劃。回到家後，煮了一些意大利麵，和臭臭一起享用。今天牠的狀態很好，我也感到很放鬆。
example_assistant: [{"color": "Yellow", "content": "午休時，莉莉邀請我週末一起去看電影，我感到非常開心。"}, {"color": "Indigo", "content": "回到家後，煮了一些意大利麵，和臭臭一起享用。今天牠的狀態很好，我也感到很放鬆。"}]

example_user: 今天沒有發生有趣的事
example_assistant: [{"color": "Gray", "content": "今天沒有發生有趣的事"}]

example_user: (使用者沒有輸入任何內容)
example_assistant: [{"color": "Gray", "content": ""}]"""


In [4]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.globals import set_verbose

set_verbose(True)

class EmotionAnalysis(BaseModel):
    color: str = Field(description="Emotion color based on the diary content")
    content: str = Field(description="Content of the diary representing the emotion")

class EmotionAnalysisList(BaseModel):
    colors: List[EmotionAnalysis]  

model = ChatOpenAI(model="gpt-3.5-turbo-0125")


parser = JsonOutputParser(pydantic_object=EmotionAnalysisList)

prompt = PromptTemplate(
    template="""你是一個專業的心理學家，你擅長分析日記中所包含的情緒。 \
                你需要根據使用者的日記內容分析其情緒，並挑選其代表的顏色以及對應的內容段落。 \
                注意：每個段落不要超過 100 字，同時不要回傳日記以外的內容。 \
                限制：你最少需要挑選一個顏色，並且最多挑選不超過兩個顏色，請挑選日記中最具有代表性的段落。 \
                以下是 8 種不同的情緒所對應的顏色: \
                1. 憤怒暴躁: Red \
                2. 快樂喜悅: Yellow \
                3. 悲傷難過: Blue \
                4. 恐懼害怕: Purple \
                5. 焦慮不安: Orange \
                6. 厭惡煩躁: Green \
                7. 平靜祥和: Indigo \
                8. 其他 (當日記不包含任何情緒或是沒有內容時): Gray \

                {format_instructions}

                {query}
            """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

diary_entry = "今天一早出門時，感覺有些不安。工作中遇到一些困難的問題，但在與同事的共同努力下最終解決了。"
output = chain.invoke({"query": diary_entry})

print(output['colors'][:2])


[{'color': 'Orange', 'content': '今天一早出門時，感覺有些不安。工作中遇到一些困難的問題，但在與同事的共同努力下最終解決了。'}]


In [20]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI, OpenAIEmbeddings



diary_to_tag_summary_template = """你是一個擅長將文字整理成摘要的專家。你需要將日記的內容整理成簡短的記憶摘要，以便未來能夠快速回憶。
針對日記的內容，請只挑選重要的事件或回憶並輸出摘要，並將其轉為 JSON 格式放置於 summary 欄位中。
注意：你只需要輸出純文字，不必輸出 markdown 語法。 \n
同時也請你為本日記輸出一個對應的 tag，以下是可以輸出的 tag 名稱以及說明：
1. 人際關係: 當日記內容主要描述和他人關係的變化時，輸出此tag
2. 學業事業: 當日記內容主要圍繞校園或職場生活或事件時，輸出此tag
3. 日常生活: 當日記內容主要為日常瑣事時，輸出此tag
4. 自我成長: 當日記內容主要表達自我的成就感或是自我實現時，輸出此tag
5. 其他: 當日記無法被歸類為以上任一tag時，輸出此tag
請根據以上 tag 說明，將輸出的 tag 放置於 tag 欄位中。

{format_instructions}
                    
以下是日記的內容：
{query}
"""

class DiarySummary(BaseModel):
    tag: str = Field(description="The tag for the diary")
    summary: str = Field(description="Summary of the given diary content")

def convert_diary_to_summary(diary_content):
    parser = JsonOutputParser(pydantic_object=DiarySummary)
    prompt = PromptTemplate(
        template=diary_to_tag_summary_template,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    model = ChatOpenAI(model="gpt-4o")
    runnable = prompt | model | parser
    summary_item = runnable.invoke({"query": diary_content})

    embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
    summary_embedding = embeddings_model.embed_query(summary_item['summary'])
    summary_item['summary_embedding'] = summary_embedding

    return summary_item

diary_content = "今天一早出門，心情很好。天氣晴朗，陽光明媚，這樣的早晨讓人感到充滿活力。工作中處理了一些複雜的代碼問題，但最終解決了。這些問題讓我花了不少時間，不過解決後的成就感讓我覺得一切都值得。莉莉在午餐時問我對於電影的期待，我們聊得很愉快。她告訴我，這部電影的導演是她非常喜歡的一位，她已經期待很久了。看著她談論電影時眼中閃爍的光芒，我感到我們之間的距離似乎又拉近了一些。下午的工作依然忙碌，但與莉莉的對話讓我一整天心情愉快。下班後，回家的路上，我依然在想著我們的約會。回到家後，我做了一些炒青菜，簡單但健康，和“臭臭”一起度過了這個平靜的晚上。牠今天的健康狀況依然很好，吃完晚餐後，我們一起玩了一會兒。看著牠活潑的樣子，我感到很欣慰。晚上的時間，我靜靜地坐在書桌前，整理了一下今天的工作進展，為明天做了一些準備。生活似乎開始變得不再那麼單調和沉悶，我感受到了一絲久違的快樂。這種感覺讓我更加期待未來，期待與莉莉的每一次相處。希望這一切都能順利進行，我的生活也能因此變得更加充實和美好。"
summary_item = convert_diary_to_summary(diary_content)
print(summary_item)

{'tag': '日常生活', 'summary': '今天一早出門心情很好，工作中解決了複雜的代碼問題，感到成就感。午餐時與莉莉愉快地聊了電影，感到彼此更親近。下午工作忙碌但心情愉快。下班後回家做了簡單的晚餐，和寵物“臭臭”度過平靜的晚上，牠健康狀況良好。晚上整理了工作進展，為明天做準備，感受到久違的快樂並期待未來。', 'summary_embedding': [0.03797202184796333, -0.0284272450953722, -0.05798405036330223, 0.01290521677583456, 0.02596104145050049, -0.0020473264157772064, 0.04461759701371193, 0.0568544901907444, -0.01684926077723503, -0.019183684140443802, -0.02855902723968029, 0.020049680024385452, -0.045107074081897736, -0.030611060559749603, -0.019193097949028015, 0.004419401288032532, -0.020614460110664368, 0.02119806595146656, -0.006589096505194902, 0.007026800885796547, 0.01414773240685463, -0.0168963260948658, 0.004231141414493322, -0.006556151434779167, -0.02407844178378582, -0.014128905721008778, -0.01645391620695591, 0.026902340352535248, 0.020294418558478355, -0.05135730281472206, -0.035072822123765945, -0.018458884209394455, 0.07530396431684494, -0.010749640874564648, -0.0032357170712202787, 0.035411689430475235, -0.0084905214607715